In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '954438', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Jan-21,"25,730","24,878",852,3.3,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Jan-21,"95,635","92,248","3,387",3.5,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Jan-21,"8,483","7,918",565,6.7,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Jan-21,"8,608","8,256",352,4.1,01007
38631,CN0100900000000,1,9,"Blount County, AL",Jan-21,"24,754","24,153",601,2.4,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Jan-21,"12,682","11,244",1438,11.3,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Jan-21,"2,462","2,132",330,13.4,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Jan-21,"6,875","5,880",995,14.5,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Jan-21,"8,271","7,188",1083,13.1,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Jan-21,"9,001","7,775",1226,13.6,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1254702,2021-04-24,Webster,West Virginia,54101.0,458,7.0
1254703,2021-04-24,Wetzel,West Virginia,54103.0,1244,29.0
1254704,2021-04-24,Wirt,West Virginia,54105.0,384,3.0
1254705,2021-04-24,Wood,West Virginia,54107.0,7620,168.0
1254706,2021-04-24,Wyoming,West Virginia,54109.0,1947,43.0
1254707,2021-04-24,Adams,Wisconsin,55001.0,1875,17.0
1254708,2021-04-24,Ashland,Wisconsin,55003.0,1315,15.0
1254709,2021-04-24,Barron,Wisconsin,55005.0,5835,77.0
1254710,2021-04-24,Bayfield,Wisconsin,55007.0,1243,19.0
1254711,2021-04-24,Brown,Wisconsin,55009.0,34643,275.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
414,04/18/2021,1001,512,146,31,4,1851,2512,173,33,37,156,95,29,2,1,293,412,32,5,5,336,139,58,13,0,609,812,55,12,13,120,68,10,8,0,244,341,22,5,6,286,157,40,6,0,541,731,52,9,9,101,50,9,2,0,165,213,12,2,2,11000
415,04/19/2021,1778,636,149,33,9,1750,2381,168,32,37,263,128,26,4,1,274,392,31,4,5,609,202,46,13,3,576,768,55,12,13,224,76,19,2,0,229,321,21,5,5,505,183,46,11,0,510,690,50,9,9,176,42,12,3,0,161,206,12,2,2,11000
416,04/20/2021,1463,492,123,29,6,1631,2220,159,32,37,253,108,26,5,0,258,371,30,4,5,476,143,48,11,1,535,711,53,12,12,208,71,17,2,0,212,297,19,4,5,389,129,25,9,0,470,636,45,9,9,137,36,7,2,0,157,202,11,2,2,11000
417,04/21/2021,1237,521,73,20,14,1494,2060,144,30,37,166,98,11,0,0,233,343,27,4,4,449,163,22,7,2,497,669,48,11,12,157,67,9,3,0,190,268,17,4,5,345,134,23,10,1,431,586,41,9,9,118,47,8,0,0,143,188,11,2,2,11000
418,04/22/2021,1053,450,10,21,17,1363,1902,117,27,36,169,88,4,4,0,208,314,22,3,4,381,122,4,11,0,462,624,41,11,12,107,60,0,1,0,166,240,13,3,4,290,128,2,4,0,394,541,33,8,8,106,35,0,1,0,133,176,9,2,2,11000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
414,04/18/2021,156,2,New York,Bronx,36005.0
415,04/19/2021,263,4,New York,Bronx,36005.0
416,04/20/2021,253,5,New York,Bronx,36005.0
417,04/21/2021,166,0,New York,Bronx,36005.0
418,04/22/2021,169,4,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
414,04/18/2021,336,13,Kings,New York,36047.0
415,04/19/2021,609,13,Kings,New York,36047.0
416,04/20/2021,476,11,Kings,New York,36047.0
417,04/21/2021,449,7,Kings,New York,36047.0
418,04/22/2021,381,11,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,6625,149


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
405,04/09/2021,724,10,New York,Queens,36081.0,2021-04-09
406,04/10/2021,564,18,New York,Queens,36081.0,2021-04-10
407,04/11/2021,388,17,New York,Queens,36081.0,2021-04-11
408,04/12/2021,722,11,New York,Queens,36081.0,2021-04-12
409,04/13/2021,670,10,New York,Queens,36081.0,2021-04-13
410,04/14/2021,618,11,New York,Queens,36081.0,2021-04-14
411,04/15/2021,548,9,New York,Queens,36081.0,2021-04-15
412,04/16/2021,547,8,New York,Queens,36081.0,2021-04-16
413,04/17/2021,394,7,New York,Queens,36081.0,2021-04-17
414,04/18/2021,286,6,New York,Queens,36081.0,2021-04-18


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
369,03/04/2021,Richmond,New York,36085.0,252,5.0,2021-03-04
370,03/05/2021,Richmond,New York,36085.0,236,5.0,2021-03-05
371,03/06/2021,Richmond,New York,36085.0,172,3.0,2021-03-06
372,03/07/2021,Richmond,New York,36085.0,148,3.0,2021-03-07
373,03/08/2021,Richmond,New York,36085.0,260,7.0,2021-03-08
374,03/09/2021,Richmond,New York,36085.0,233,1.0,2021-03-09
375,03/10/2021,Richmond,New York,36085.0,260,5.0,2021-03-10
376,03/11/2021,Richmond,New York,36085.0,210,5.0,2021-03-11
377,03/12/2021,Richmond,New York,36085.0,245,1.0,2021-03-12
378,03/13/2021,Richmond,New York,36085.0,177,5.0,2021-03-13


In [46]:
print (data_date_dt)

2021-04-25 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1251555,1001.0,107.0
1251556,1003.0,305.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,8279


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1254557,51690.0,76.0
1254558,51115.0,12.0
1254559,51117.0,64.0
1254560,51119.0,20.0
1254561,51121.0,89.0
1254562,51125.0,12.0
1254563,51127.0,16.0
1254564,51700.0,221.0
1254565,51710.0,248.0
1254566,51131.0,35.0


In [50]:
covid_death.shape

(3252, 2)

In [51]:
print (County_covid_death)

{1001.0: 107.0, 1003.0: 305.0, 1005.0: 56.0, 1007.0: 62.0, 1009.0: 134.0, 1011.0: 40.0, 1013.0: 69.0, 1015.0: 312.0, 1017.0: 121.0, 1019.0: 45.0, 1021.0: 111.0, 1023.0: 24.0, 1025.0: 60.0, 1027.0: 56.0, 1029.0: 41.0, 1031.0: 115.0, 1033.0: 134.0, 1035.0: 28.0, 1037.0: 28.0, 1039.0: 116.0, 1041.0: 57.0, 1043.0: 191.0, 1045.0: 111.0, 1047.0: 150.0, 1049.0: 184.0, 1051.0: 204.0, 1053.0: 75.0, 1055.0: 352.0, 1057.0: 62.0, 1059.0: 82.0, 1061.0: 75.0, 1063.0: 34.0, 1065.0: 75.0, 1067.0: 44.0, 1069.0: 280.0, 1071.0: 110.0, 1073.0: 1498.0, 1075.0: 34.0, 1077.0: 236.0, 1079.0: 97.0, 1081.0: 168.0, 1083.0: 150.0, 1085.0: 53.0, 1087.0: 49.0, 1089.0: 496.0, 1091.0: 64.0, 1093.0: 100.0, 1095.0: 221.0, 1097.0: 801.0, 1099.0: 40.0, 1101.0: 575.0, 1103.0: 277.0, 1105.0: 26.0, 1107.0: 59.0, 1109.0: 76.0, 1111.0: 41.0, 1113.0: 38.0, 1117.0: 246.0, 1115.0: 237.0, 1119.0: 32.0, 1121.0: 174.0, 1123.0: 149.0, 1125.0: 448.0, 1127.0: 278.0, 1129.0: 39.0, 1131.0: 27.0, 1133.0: 72.0, 2013.0: 2.0, 2016.0: 0.0, 2

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
1209654,2021-04-11,New Haven,Connecticut,9009.0,84116,2015.0,2021-04-11,-14 days,NaN
1212901,2021-04-12,New Haven,Connecticut,9009.0,84979,2019.0,2021-04-12,-13 days,863.0
1216148,2021-04-13,New Haven,Connecticut,9009.0,85325,2024.0,2021-04-13,-12 days,346.0
1219395,2021-04-14,New Haven,Connecticut,9009.0,85768,2028.0,2021-04-14,-11 days,443.0
1222642,2021-04-15,New Haven,Connecticut,9009.0,85999,2030.0,2021-04-15,-10 days,231.0
1225888,2021-04-16,New Haven,Connecticut,9009.0,86260,2032.0,2021-04-16,-9 days,261.0
1229135,2021-04-17,New Haven,Connecticut,9009.0,86260,2032.0,2021-04-17,-8 days,0.0
1232384,2021-04-18,New Haven,Connecticut,9009.0,86260,2032.0,2021-04-18,-7 days,0.0
1235631,2021-04-19,New Haven,Connecticut,9009.0,86954,2039.0,2021-04-19,-6 days,694.0
1238878,2021-04-20,New Haven,Connecticut,9009.0,87196,2040.0,2021-04-20,-5 days,242.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,3985.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
369,03/04/2021,Richmond,New York,36085.0,252,5.0,2021-03-04
370,03/05/2021,Richmond,New York,36085.0,236,5.0,2021-03-05
371,03/06/2021,Richmond,New York,36085.0,172,3.0,2021-03-06
372,03/07/2021,Richmond,New York,36085.0,148,3.0,2021-03-07
373,03/08/2021,Richmond,New York,36085.0,260,7.0,2021-03-08
374,03/09/2021,Richmond,New York,36085.0,233,1.0,2021-03-09
375,03/10/2021,Richmond,New York,36085.0,260,5.0,2021-03-10
376,03/11/2021,Richmond,New York,36085.0,210,5.0,2021-03-11
377,03/12/2021,Richmond,New York,36085.0,245,1.0,2021-03-12
378,03/13/2021,Richmond,New York,36085.0,177,5.0,2021-03-13


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,70.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-04-25 20:39:32.426612
4


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 107.0
1003.0 305.0
1005.0 56.0
1007.0 62.0
1009.0 134.0
1011.0 40.0
1013.0 69.0
1015.0 312.0
1017.0 121.0
1019.0 45.0
1021.0 111.0
1023.0 24.0
1025.0 60.0
1027.0 56.0
1029.0 41.0
1031.0 115.0
1033.0 134.0
1035.0 28.0
1037.0 28.0
1039.0 116.0
1041.0 57.0
1043.0 191.0
1045.0 111.0
1047.0 150.0
1049.0 184.0
1051.0 204.0
1053.0 75.0
1055.0 352.0
1057.0 62.0
1059.0 82.0
1061.0 75.0
1063.0 34.0
1065.0 75.0
1067.0 44.0
1069.0 280.0
1071.0 110.0
1073.0 1498.0
1075.0 34.0
1077.0 236.0
1079.0 97.0
1081.0 168.0
1083.0 150.0
1085.0 53.0
1087.0 49.0
1089.0 496.0
1091.0 64.0
1093.0 100.0
1095.0 221.0
1097.0 801.0
1099.0 40.0
1101.0 575.0
1103.0 277.0
1105.0 26.0
1107.0 59.0
1109.0 76.0
1111.0 41.0
1113.0 38.0
1117.0 246.0
1115.0 237.0
1119.0 32.0
1121.0 174.0
1123.0 149.0
1125.0 448.0
1127.0 278.0
1129.0 39.0
1131.0 27.0
1133.0 72.0
2013.0 2.0
2016.0 0.0
2020.0 169.0
2050.0 20.0
2997.0 0.0
2068.0 0.0
2070.0 2.0
2090.0 36.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 1.0
2150.0 5.0
2158.0 4.0
217

24047.0 97.0
25001.0 461.0
25003.0 282.0
25005.0 1683.0
25007.0 0.0
25009.0 2340.0
25011.0 112.0
25013.0 1485.0
25015.0 290.0
25017.0 3709.0
25019.0 0.0
25021.0 1768.0
25023.0 1405.0
25025.0 1811.0
nan 12.0
25027.0 2186.0
26001.0 26.0
26003.0 6.0
26005.0 126.0
26007.0 55.0
26009.0 24.0
26011.0 28.0
26013.0 34.0
26015.0 60.0
26017.0 319.0
26019.0 32.0
26021.0 265.0
26023.0 118.0
26025.0 267.0
26027.0 72.0
26029.0 18.0
26031.0 41.0
26033.0 48.0
26035.0 71.0
26037.0 80.0
26039.0 11.0
26041.0 87.0
26043.0 73.0
26045.0 180.0
26047.0 41.0
26049.0 850.0
26051.0 46.0
26053.0 50.0
26055.0 86.0
26057.0 114.0
26059.0 82.0
26061.0 40.0
26063.0 71.0
26065.0 341.0
26067.0 81.0
26069.0 69.0
26071.0 51.0
26073.0 83.0
26075.0 286.0
26077.0 351.0
26079.0 20.0
26081.0 777.0
26083.0 2.0
26085.0 13.0
26087.0 169.0
26089.0 17.0
26091.0 155.0
26093.0 167.0
26095.0 2.0
26097.0 3.0
26099.0 2181.0
26101.0 23.0
26103.0 58.0
26105.0 30.0
26107.0 27.0
26109.0 41.0
26111.0 76.0
26113.0 23.0
26115.0 235.0
26117.0 10

48193.0 26.0
48195.0 24.0
48197.0 12.0
48199.0 110.0
48201.0 6144.0
48203.0 106.0
48205.0 2.0
48207.0 27.0
48209.0 239.0
48211.0 2.0
48213.0 184.0
48215.0 2810.0
48217.0 107.0
48219.0 102.0
48221.0 127.0
48223.0 109.0
48225.0 52.0
48227.0 101.0
48229.0 11.0
48231.0 166.0
48233.0 71.0
48235.0 1.0
48237.0 16.0
48239.0 31.0
48241.0 88.0
48243.0 5.0
48245.0 419.0
48247.0 16.0
48249.0 100.0
48251.0 365.0
48253.0 55.0
48255.0 36.0
48257.0 255.0
48259.0 72.0
48261.0 2.0
48263.0 2.0
48265.0 87.0
48267.0 6.0
48269.0 0.0
48271.0 6.0
48273.0 93.0
48275.0 17.0
48283.0 28.0
48277.0 154.0
48279.0 82.0
48281.0 32.0
48285.0 76.0
48287.0 39.0
48289.0 42.0
48291.0 179.0
48293.0 76.0
48295.0 12.0
48297.0 22.0
48299.0 51.0
48301.0 0.0
48303.0 783.0
48305.0 23.0
48313.0 29.0
48315.0 33.0
48317.0 18.0
48319.0 6.0
48321.0 103.0
48323.0 352.0
48307.0 19.0
48309.0 470.0
48311.0 5.0
48325.0 103.0
48327.0 9.0
48329.0 266.0
48331.0 47.0
48333.0 21.0
48335.0 30.0
48337.0 71.0
48339.0 553.0
48341.0 72.0
48343.0 33.

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,6625,149


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 6625}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 57.0
1001 153.0
1009 87.0
1013 24.0
1015 116.0
1017 30.0
1031 55.0
1033 47.0
1039 44.0
1043 86.0
1045 19.0
1051 38.0
1055 72.0
1067 11.0
1069 77.0
1071 46.0
1077 102.0
1079 30.0
1083 74.0
1089 462.0
1095 120.0
1097 2819.0
1103 49.0
1111 20.0
1113 51.0
1115 102.0
1117 152.0
1121 77.0
2261 27.0
4021 693.0
5009 29.0
5011 12.0
5033 38.0
5037 9.0
5045 107.0
5047 7.0
5051 37.0
5053 4.0
5063 16.0
5083 16.0
5085 126.0
5087 6.0
5115 78.0
5117 1.0
5121 24.0
5125 66.0
5131 61.0
5145 55.0
6007 190.0
6017 239.0
6023 199.0
6027 10.0
6061 626.0
6079 348.0
6089 270.0
6093 88.0
8019 12.0
8039 113.0
8047 13.0
8051 35.0
8057 4.0
8065 52.0
8067 191.0
8071 32.0
8075 49.0
8077 322.0
8083 21.0
8085 43.0
8093 60.0
8099 20.0
8101 917.0
8103 11.0
8119 96.0
8121 4.0
9005 624.0
9007 458.0
9011 535.0
9015 358.0
10001 748.0
12019 566.0
12033 515.0
12089 180.0
12101 2066.0
12109 561.0
12113 181.0
13015 308.0
13039 45.0
13045 133.0
13047 198.0
13057 484.0
13073 244.0
13077 196.0
13085 47.0
13113 157.0
13115 105

39081 184.0
39083 96.0
39085 484.0
39089 246.0
39091 65.0
39093 645.0
39099 445.0
39103 334.0
39107 73.0
39109 120.0
39111 7.0
39113 996.0
39117 59.0
39119 92.0
39123 90.0
39125 20.0
39127 50.0
39133 360.0
39135 53.0
39137 65.0
39139 212.0
39141 106.0
39143 139.0
39147 135.0
39149 86.0
39151 562.0
39153 1376.0
39155 334.0
39157 93.0
39159 111.0
39161 29.0
39165 308.0
39167 79.0
39169 190.0
39171 142.0
39175 50.0
40013 28.0
40015 25.0
40017 396.0
40019 62.0
40021 30.0
40027 699.0
40031 351.0
40033 4.0
40035 15.0
40037 67.0
40045 1.0
40047 48.0
40049 47.0
40051 75.0
40053 0.0
40065 35.0
40071 31.0
40075 3.0
40081 37.0
40083 53.0
40085 18.0
40087 74.0
40093 4.0
40103 3.0
40105 15.0
40113 49.0
40117 16.0
40123 30.0
40125 103.0
40129 2.0
40131 137.0
40133 27.0
40137 31.0
40145 109.0
40147 80.0
40149 2.0
40153 12.0
41007 52.0
41009 100.0
41017 810.0
41029 584.0
41035 500.0
41037 11.0
41043 306.0
41053 173.0
41061 21.0
41065 71.0
41071 168.0
42001 445.0
42005 263.0
42007 685.0
42009 222.0
420

56005 42.0
56017 3.0
56023 7.0
56027 1.0
56037 74.0
56041 22.0
1007 25.0
1037 65.0
1053 35.0
1101 185.0
1107 5.0
1127 85.0
1129 27.0
2020 739.0
2100 0.0
2130 24.0
4005 265.0
4011 3.0
5001 8.0
5013 1.0
5021 18.0
5027 6.0
5119 315.0
5129 3.0
5135 27.0
5141 19.0
5149 9.0
6005 34.0
6015 48.0
6051 7.0
6055 183.0
6087 471.0
6097 237.0
6103 86.0
6111 467.0
8033 12.0
8037 195.0
8045 134.0
8053 5.0
8055 23.0
8073 6.0
8087 43.0
8109 21.0
8111 5.0
8117 95.0
8125 18.0
9001 4096.0
9009 3985.0
12003 64.0
12007 40.0
12023 103.0
12055 392.0
12057 6236.0
12061 304.0
12065 37.0
12081 1529.0
12085 357.0
12087 175.0
12099 5142.0
12115 1363.0
12129 91.0
13007 0.0
13011 32.0
13023 8.0
13029 61.0
13031 79.0
13053 168.0
13075 14.0
13079 4.0
13111 32.0
13119 30.0
13129 66.0
13139 295.0
13191 17.0
13199 30.0
13209 11.0
13213 113.0
13217 281.0
13229 46.0
13247 168.0
13251 24.0
13253 9.0
13263 2.0
13277 36.0
13283 2.0
13319 6.0
13321 30.0
15001 90.0
15007 8.0
16031 11.0
16035 6.0
16037 3.0
16049 11.0
16057 91.0
1

34039 2902.0
35005 93.0
35019 0.0
35023 4.0
35033 0.0
35045 371.0
35051 16.0
35053 50.0
36047 667.0
36081 553.0
36087 981.0
37083 137.0
38005 8.0
40001 6.0
40139 12.0
41049 17.0
45009 13.0
45033 60.0
45053 39.0
45075 157.0
45089 59.0
47007 23.0
47061 21.0
48003 1.0
48013 78.0
48017 4.0
48025 30.0
48069 16.0
48079 14.0
48095 0.0
48101 0.0
48103 6.0
48113 3078.0
48117 39.0
48153 18.0
48169 22.0
48177 27.0
48195 26.0
48201 6952.0
48207 2.0
48249 54.0
48255 178.0
48271 3.0
48279 69.0
48301 0.0
48341 56.0
48357 30.0
48369 8.0
48371 99.0
48375 62.0
48389 18.0
48419 33.0
48435 0.0
48437 12.0
48463 25.0
48465 39.0
48501 3.0
51027 26.0
51051 7.0
51105 23.0
51183 24.0
51195 65.0
51640 50.0
51710 491.0
51720 6.0
54005 86.0
54015 37.0
54043 46.0
1047 13.0
1087 32.0
1119 8.0
2016 30.0
2070 17.0
j 2164
2180 5.0
4017 139.0
4023 40.0
4027 112.0
5017 2.0
5123 30.0
6011 32.0
8023 2.0
12043 16.0
12067 6.0
13043 6.0
13061 1.0
13065 15.0
13161 16.0
13163 25.0
13205 29.0
13243 5.0
13271 20.0
13273 7.0
20055

In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 1819
36061 2545
36047 6625 6625
36081 5702
36005 3072


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 153.0, 1003.0: 207.0, 1005.0: 49.0, 1007.0: 25.0, 1009.0: 87.0, 1011.0: 11.0, 1013.0: 24.0, 1015.0: 116.0, 1017.0: 30.0, 1019.0: 13.0, 1021.0: 92.0, 1023.0: 11.0, 1025.0: 35.0, 1027.0: 29.0, 1029.0: 16.0, 1031.0: 55.0, 1033.0: 47.0, 1035.0: 8.0, 1037.0: 65.0, 1039.0: 44.0, 1041.0: 20.0, 1043.0: 86.0, 1045.0: 19.0, 1047.0: 13.0, 1049.0: 40.0, 1051.0: 38.0, 1053.0: 35.0, 1055.0: 72.0, 1057.0: 25.0, 1059.0: 22.0, 1061.0: 7.0, 1063.0: 13.0, 1065.0: 15.0, 1067.0: 11.0, 1069.0: 77.0, 1071.0: 46.0, 1073.0: 559.0, 1075.0: 13.0, 1077.0: 102.0, 1079.0: 30.0, 1081.0: 198.0, 1083.0: 74.0, 1085.0: 16.0, 1087.0: 32.0, 1089.0: 462.0, 1091.0: 5.0, 1093.0: 22.0, 1095.0: 120.0, 1097.0: 2819.0, 1099.0: 32.0, 1101.0: 185.0, 1103.0: 49.0, 1105.0: 2.0, 1107.0: 5.0, 1109.0: 16.0, 1111.0: 20.0, 1113.0: 51.0, 1115.0: 102.0, 1117.0: 152.0, 1119.0: 8.0, 1121.0: 77.0, 1123.0: 60.0, 1125.0: 240.0, 1127.0: 85.0, 1129.0: 27.0, 1131.0: 21.0, 1133.0: 10.0, 2013.0: 0.0, 2016.0: 30.0, 2020.0: 739.0, 2050.0: 49.

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 954438
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 75.0 38921 192.6980293414866
1001 107.0 55869 191.51944727845498
1009 134.0 57826 231.7296717739425
1013 69.0 19448 354.79226655697244
1015 312.0 113605 274.63579948065666
1017 121.0 33254 363.86600108257653
1031 115.0 52342 219.7088380268236
1033 134.0 55241 242.5734508788762
1039 116.0 37049 313.0988690652919
1043 191.0 83768 228.01069620857606
1045 111.0 49172 225.738225006101
1051 204.0 81209 251.2036843207034
1055 352.0 102268 344.1936871748739
1067 44.0 17205 255.73961057832025
1069 280.0 105882 264.4453259288642
1071 110.0 51626 213.07093325068763
1077 236.0 92729 254.5050631409807
1079 97.0 32924 294.617907909124
1083 150.0 98915 151.64535206995905
1089 496.0 372909 133.0083210649247
1095 221.0 96774 228.36712340091347
1097 801.0 413210 193.848164371627
1103 277.0 119679 231.45246868707122
1111 41.0 22722 180.4418625121028
1113 38.0 57961 65.56132571901796
1115 237.0 89512 264.76896952363927
1117 246.0 217702 112.9985025401696
1121 174.0 79978 217.55982895296208
2261 2.0 

29087 13.0 4403 295.25323642970704
29089 7.0 10001 69.99300069993001
29091 43.0 40117 107.18647954732407
29097 127.0 121328 104.67493076618752
29099 240.0 225081 106.6282804856918
29103 2.0 3959 50.51780752715332
29105 68.0 35723 190.35355373288917
29107 56.0 32708 171.21193591781827
29109 81.0 38355 211.1849824012515
29113 40.0 59013 67.78167522410317
29115 13.0 11920 109.06040268456375
29121 15.0 15117 99.22603691208573
29125 8.0 8697 91.98574220995745
29127 44.0 28530 154.22362425517
29129 2.0 3617 55.294442908487696
29131 54.0 25619 210.78106093134005
29137 11.0 8644 127.25590004627487
29139 11.0 11551 95.22985022941737
29141 43.0 20627 208.46463373248653
29145 74.0 58236 127.06916683838175
29151 15.0 13615 110.17260374586854
29157 30.0 19136 156.77257525083613
29159 78.0 42339 184.2273081556012
29161 128.0 44573 287.16936261862566
29167 40.0 32149 124.4206662726679
29171 4.0 4696 85.17887563884157
29173 12.0 10309 116.40314288485789
29177 27.0 23018 117.29950473542445
29183 477.0 

40093 21.0 7629 275.2654345261502
40103 19.0 11131 170.69445692210942
40105 18.0 10076 178.64231838030966
40113 72.0 46963 153.3121819304559
40117 47.0 16376 287.00537371763556
40123 72.0 38284 188.06812245324417
40125 123.0 72592 169.440158695173
40129 12.0 3583 334.9148758024002
40131 170.0 92459 183.86528082717746
40133 69.0 24258 284.4422458570368
40137 101.0 43143 234.1051850821686
40145 112.0 81289 137.78001943682418
40147 107.0 51527 207.6581209851146
40149 21.0 10916 192.37816049835104
40153 33.0 20211 163.2774231853941
41007 8.0 40224 19.88862370723946
41009 26.0 52354 49.66191694999427
41017 73.0 197692 36.926127511482505
41029 129.0 220944 58.38583532478818
41035 59.0 68238 86.46208857234971
41037 7.0 7869 88.95666539585716
41043 64.0 129749 49.3260063661377
41053 52.0 86085 60.40541325434164
41061 23.0 26835 85.70896217626235
41065 28.0 26682 104.93965969567499
41071 75.0 107100 70.0280112044818
42001 169.0 103009 164.06333427176267
42005 130.0 64735 200.81872248397312
4200

13171 56.0 19077 293.5472034386958
13177 58.0 29992 193.38490264070418
13181 26.0 7921 328.2413836636788
13185 205.0 117406 174.6077713234417
13195 56.0 29880 187.41633199464525
13201 13.0 5718 227.35222105631337
13219 72.0 40280 178.7487586891758
13221 29.0 15259 190.05177272429387
13223 212.0 168667 125.69145120266562
13237 57.0 22119 257.69700257697
13241 49.0 17137 285.93102643403165
13255 186.0 66703 278.84802782483547
13257 81.0 25925 312.4397299903568
13275 123.0 44451 276.7091853951542
13281 54.0 12037 448.6167649746614
13285 221.0 69922 316.0664740711078
13291 73.0 24511 297.8254661172535
13293 126.0 26320 478.72340425531917
13305 84.0 29927 280.68299528853544
13317 27.0 9777 276.15833077631174
16005 106.0 87808 120.71793002915452
16007 2.0 6125 32.6530612244898
16009 9.0 9298 96.79500967950096
16011 73.0 46811 155.94625194932814
16013 18.0 23021 78.18947917119152
16015 2.0 7831 25.539522410930914
16019 168.0 119062 141.10295476306462
16027 292.0 229849 127.03992621242641
1602

29213 92.0 55928 164.49721069947074
29225 53.0 39592 133.86542735906244
30011 5.0 1252 399.36102236421726
30019 7.0 1690 414.2011834319527
30037 2.0 821 243.60535931790497
30039 2.0 3379 59.18910920390648
30055 1.0 1664 60.09615384615385
30057 8.0 8600 93.0232558139535
30065 10.0 4633 215.84286639326572
30077 10.0 6890 145.1378809869376
30079 2.0 1077 185.70102135561746
30087 48.0 8937 537.0929842228936
30089 15.0 12113 123.8338974655329
30093 85.0 34915 243.44837462408705
30097 7.0 3737 187.31602890018732
30109 6.0 969 619.1950464396285
31009 0.0 465 0.0
31013 9.0 10783 83.46471297412594
31015 3.0 1919 156.3314226159458
31017 2.0 2955 67.68189509306261
31029 5.0 3924 127.420998980632
31045 18.0 8589 209.570380719525
31049 2.0 1794 111.48272017837235
31055 696.0 571327 121.82165379896277
31057 4.0 1693 236.26698168930892
31061 4.0 2979 134.2732460557234
31063 1.0 2627 38.06623524933384
31069 1.0 1837 54.43658138268917
31071 3.0 1969 152.36160487557137
31073 5.0 1990 251.25628140703517


22007 36.0 21891 164.45114430587913
22033 791.0 440059 179.74862461624463
22037 112.0 19135 585.3148680428534
22047 98.0 32511 301.4364368982806
22051 888.0 432493 205.32124219351527
22059 36.0 14892 241.74053182917004
22075 26.0 23197 112.08345906798293
22077 73.0 21730 335.9410952600092
22085 60.0 23884 251.2142019762184
22095 137.0 42837 319.81698064757103
22119 111.0 38340 289.51486697965566
22127 43.0 13904 309.26352128883775
23021 4.0 16785 23.830801310694074
24001 206.0 70416 292.5471483753692
24019 50.0 31929 156.59745059350433
24047 97.0 52276 185.55360012242713
25017 3709.0 1611699 230.1298195258544
26001 26.0 10405 249.87986544930322
26033 48.0 37349 128.51749712174356
26035 71.0 30950 229.40226171243944
26049 850.0 405813 209.4560795243129
26069 69.0 25127 274.60500656664146
26095 2.0 6229 32.10788248515011
26129 36.0 20997 171.45306472353192
26153 5.0 8094 61.774153694094394
27031 0.0 5463 0.0
27053 1698.0 1265843 134.13985778647114
27087 8.0 5527 144.74398407816176
27103 

13245 494.0 202518 243.9289347119762
13249 5.0 5257 95.11128019783146
13279 112.0 26830 417.4431606410734
13287 38.0 7985 475.8922980588604
13315 47.0 8635 544.2964678633468
15009 49.0 167417 29.26823440869207
16003 4.0 4294 93.15323707498835
16047 23.0 15179 151.52513340799788
16063 11.0 5366 204.9944092433843
17003 11.0 5761 190.9390730775907
17019 148.0 209689 70.58071715731393
17151 3.0 4177 71.82188173330141
17153 7.0 5335 131.20899718837865
18001 54.0 35777 150.9349582133773
18027 101.0 33351 302.83949506761417
18087 72.0 39614 181.7539253799162
18105 168.0 148431 113.18390363199062
19021 40.0 19620 203.87359836901123
19051 24.0 9000 266.6666666666667
20067 18.0 7150 251.74825174825176
20075 1.0 2539 39.38558487593541
21011 20.0 12500 160.0
21041 19.0 10631 178.7226037061424
21047 104.0 70461 147.59938121797876
21099 45.0 19035 236.4066193853428
21115 33.0 22188 148.72904272579774
21143 19.0 8210 231.42509135200976
21165 16.0 6489 246.57112035752814
21169 26.0 10071 258.167014199

37155 227.0 130625 173.7799043062201
38085 11.0 4230 260.04728132387703
42101 3445.0 1584064 217.47858672376873
45067 103.0 30657 335.97547052875365
46007 9.0 3365 267.4591381872214
46041 27.0 5892 458.2484725050916
46071 14.0 3344 418.66028708133973
48109 10.0 2171 460.6172270842929
48141 2643.0 839238 314.9285423205336
48163 49.0 20306 241.30798778686102
48261 2.0 404 495.0495049504951
51595 42.0 5346 785.6341189674524
54059 39.0 23424 166.49590163934425
1005 56.0 24686 226.84922628210322
1063 34.0 8111 419.1838244359512
1105 26.0 8923 291.38182225708846
2050 20.0 18386 108.77841836179702
4001 426.0 71887 592.5967142877016
5077 34.0 8857 383.8771593090211
6019 1662.0 999101 166.34954824387123
6107 832.0 466195 178.46609251493473
12047 25.0 14428 173.27418907679512
13193 29.0 12947 223.99011353981618
13239 2.0 2299 86.9943453675511
13259 23.0 6621 347.37954991693096
21119 23.0 14806 155.34242874510332
21147 30.0 17231 174.10481109627995
21159 20.0 11195 178.6511835640911
21237 5.0 715

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(2052.0, 854757, 240.06822991797668)

In [91]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 6750.704832554059,
 'ALABAMA': 17949.72941708917,
 'ALASKA': 715.1431117727542,
 'ARIZONA': 4566.954697681165,
 'ARKANSAS': 16703.3836501087,
 'CALIFORNIA': 5804.177030044457,
 'COLORADO': 7371.225467265195,
 'CONNECTICUT': 1561.532422711554,
 'DELAWARE': 531.7861990625983,
 'FLORIDA': 12624.261624259467,
 'GEORGIA': 42215.68701883166,
 'IDAHO': 4931.174899068201,
 'ILLINOIS': 22751.61647281485,
 'INDIANA': 19270.773907146413,
 'IOWA': 24445.110613825298,
 'KANSAS': 27048.760905073606,
 'KENTUCKY': 20313.93330278712,
 'LOUISIANA': 16957.761422363572,
 'MAINE': 794.5838679169763,
 'MARYLAND': 3425.932159072155,
 'MASSACHUSETTS': 2933.0901219050543,
 'MICHIGAN': 14194.631358730694,
 'MINNESOTA': 12244.341068925136,
 'MISSISSIPPI': 23990.26056988428,
 'MISSOURI': 16989.453891064906,
 'MONTANA': 11262.797070949244,
 'NEBRASKA': 13723.518054651973,
 'NEVADA': 2147.1805511540206,
 'NEW HAMPSHIRE': 789.8034188257014,
 'NEW JERSEY': 5551.834753667902,
 'NEW YORK': 9009.312624084

In [92]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [93]:
Covid_death_capita[72049]

nan

In [94]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [95]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 954437.9999999999, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [96]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [97]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76274734.0 1000000 10122.644282181305
ALABAMA 954437.9999999999 76274734.0 1000000 12513.160648977155
ALASKA 231145.0 76274734.0 1000000 3030.4268252184265
ARIZONA 1839932.0 76274734.0 1000000 24122.43089566199
ARKANSAS 830467.0000000003 76274734.0 1000000 10887.838691118874
CALIFORNIA 11847711.0 76274734.0 1000000 155329.43058182282
COLORADO 1337805.0 76274734.0 1000000 17539.294204552716
CONNECTICUT 874974.0 76274734.0 1000000 11471.347772907344
DELAWARE 239009.0 76274734.0 1000000 3133.527807517493
FLORIDA 3892551.999999999 76274734.0 1000000 51033.30809387023
GEORGIA 1928702.999999999 76274734.0 1000000 25286.263207420678
IDAHO 340742.0000000001 76274734.0 1000000 4467.298437251845
ILLINOIS 2987496.0000000005 76274734.0 1000000 39167.5702205661
INDIANA 1602975.9999999995 76274734.0 1000000 21015.818947333195
IOWA 699741.0 76274734.0 1000000 9173.955296913917
KANSAS 401103.0000000001 76274734.0 1000000 5258.661406803465
KENTUCKY 1416013.0000000002 76274734.0 1000

In [98]:
print (Covid_state)

{'NEW MEXICO': 2863.0, 'ALABAMA': 7250.0, 'ALASKA': 2240.0, 'ARIZONA': 8572.0, 'ARKANSAS': 2492.0, 'CALIFORNIA': 30506.0, 'COLORADO': 21396.0, 'CONNECTICUT': 13153.0, 'DELAWARE': 4347.0, 'FLORIDA': 79751.0, 'GEORGIA': 18705.0, 'IDAHO': 3050.0, 'ILLINOIS': 39559.0, 'INDIANA': 15303.0, 'IOWA': 5747.0, 'KANSAS': 3089.0, 'KENTUCKY': 7787.0, 'LOUISIANA': 6963.0, 'MAINE': 5654.0, 'MARYLAND': 15623.0, 'MASSACHUSETTS': 19491.0, 'MICHIGAN': 91119.0, 'MINNESOTA': 24628.0, 'MISSISSIPPI': 2921.0, 'MISSOURI': 7723.0, 'MONTANA': 1907.0, 'NEBRASKA': 3727.0, 'NEVADA': 4823.0, 'NEW HAMPSHIRE': 4731.0, 'NEW JERSEY': 42750.0, 'NEW YORK': 57602.0, 'NORTH CAROLINA': 28665.0, 'NORTH DAKOTA': 1921.0, 'OHIO': 23985.0, 'OKLAHOMA': 5608.0, 'OREGON': 9362.0, 'PENNSYLVANIA': 58639.0, 'RHODE ISLAND': 5349.0, 'SOUTH CAROLINA': 11537.0, 'SOUTH DAKOTA': 2290.0, 'TENNESSEE': 18118.0, 'TEXAS': 46226.0, 'UTAH': 5196.0, 'VERMONT': 1277.0, 'VIRGINIA': 18070.0, 'WASHINGTON': 18852.0, 'WEST VIRGINIA': 4984.0, 'WISCONSIN': 1

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [99]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [100]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [101]:
Total_covid_death_cap


{'NEW MEXICO': 6750.704832554059,
 'ALABAMA': 17949.72941708917,
 'ALASKA': 715.1431117727542,
 'ARIZONA': 4566.954697681165,
 'ARKANSAS': 16703.3836501087,
 'CALIFORNIA': 5804.177030044457,
 'COLORADO': 7371.225467265195,
 'CONNECTICUT': 1561.532422711554,
 'DELAWARE': 531.7861990625983,
 'FLORIDA': 12624.261624259467,
 'GEORGIA': 42215.68701883166,
 'IDAHO': 4931.174899068201,
 'ILLINOIS': 22751.61647281485,
 'INDIANA': 19270.773907146413,
 'IOWA': 24445.110613825298,
 'KANSAS': 27048.760905073606,
 'KENTUCKY': 20313.93330278712,
 'LOUISIANA': 16957.761422363572,
 'MAINE': 794.5838679169763,
 'MARYLAND': 3425.932159072155,
 'MASSACHUSETTS': 2933.0901219050543,
 'MICHIGAN': 14194.631358730694,
 'MINNESOTA': 12244.341068925136,
 'MISSISSIPPI': 23990.26056988428,
 'MISSOURI': 16989.453891064906,
 'MONTANA': 11262.797070949244,
 'NEBRASKA': 13723.518054651973,
 'NEVADA': 2147.1805511540206,
 'NEW HAMPSHIRE': 789.8034188257014,
 'NEW JERSEY': 5551.834753667902,
 'NEW YORK': 9009.312624084

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [102]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [103]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [104]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [105]:
#print (Medicaid_demand)

In [106]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [107]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [108]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [109]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)